### **ML-7: Оптимизация гиперпараметров. Практика**

#### Прогнозирование биологического ответа

*Основана на соревновании Kaggle: Predicting a Biological Response (https://www.kaggle.com/c/bioresponse)*

Необходимо предсказать биологический ответ молекул (столбец 'Activity') по их химическому составу (столбцы D1-D1776).

Данные представлены в формате CSV.  Каждая строка представляет молекулу. 

- Первый столбец Activity содержит экспериментальные данные, описывающие фактический биологический ответ [0, 1]; 
- Остальные столбцы D1-D1776 представляют собой молекулярные дескрипторы — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов.

Предварительная обработка не требуется, данные уже закодированы и нормализованы.

В качестве метрики будем использовать F1-score.

Необходимо обучить две модели: логистическую регрессию и случайный лес. Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. Важно использовать все четыре метода (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna) хотя бы по разу, максимальное количество итераций не должно превышать 50.

##### 1. Загрузка данных

In [1]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных

from sklearn import linear_model #линейные моделиё
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn.model_selection import train_test_split #сплитование выборки

In [2]:
data = pd.read_csv('_train_sem09.csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = data.drop(['Activity'], axis=1)
y = data['Activity']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 1, test_size = 0.2)

##### 2. Базовые модели

In [30]:
random_state = 42

In [67]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(max_iter = 50)

#Обучаем модель
log_reg.fit(X_train, y_train)

#Выводим значения метрики 
y_test_pred = log_reg.predict(X_test)
print('Логистическая регрессия\nf1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Логистическая регрессия
f1_score на тестовом наборе: 0.79


/home/director/anaconda3/envs/DS/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [68]:
#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=random_state)

#Обучаем модель
rf.fit(X_train, y_train)

#Выводим значения метрики 
y_test_pred = rf.predict(X_test)
print('Случайный лес\nf1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Случайный лес
f1_score на тестовом наборе: 0.81


##### 3. Оптимизация гиперпараметров. Логистическая регрессия

3.1 Логистическая регрессия. GridSearchCV

In [26]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = [
              {'penalty': ['l1', 'l2'] , # тип регуляризации
              'solver': ['saga'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регуляризации

              {'penalty': ['l2'] , # тип регуляризации
              'solver': ['lbfgs'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регуляризации

              {'penalty': ['none'] , # тип регуляризации
              'solver': ['saga']}, # алгоритм оптимизации
]
grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=random_state, max_iter=50), 
    param_grid=param_grid, 
    cv=5,
    n_jobs = -1
)  
 
# %time - замеряет время выполнения
%time grid_search.fit(X_train, y_train) 

In [77]:
y_test_pred = grid_search.predict(X_test)
y_train_pred = grid_search.predict(X_train)

print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print('f1 на кросс-валидации: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))

Наилучшие значения гиперпараметров: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
f1_score на тестовом наборе: 0.79
f1 на кросс-валидации: 0.85


3.2 Логистическая регрессия. RandomizedSearchCV

In [31]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=random_state, max_iter=50), 
    param_distributions=param_grid, 
    cv=5,
    scoring='f1',
    n_jobs = -1
)  
 
random_search.fit(X_train, y_train) 

In [80]:
y_test_pred = random_search.predict(X_test)
y_train_pred = random_search.predict(X_train)

print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print('f1 на кросс-валидации: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))

Наилучшие значения гиперпараметров: {'solver': 'saga', 'penalty': 'l1', 'C': 0.9}
f1_score на тестовом наборе: 0.79
f1 на кросс-валидации: 0.85


3.3 Логистическая регрессия. HYPEROPT

In [10]:
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials

In [11]:
space = hp.choice('space', 
        [{'penalty': hp.choice('penalty', ['l1', 'l2']),
          'solver': 'saga',
          'C': hp.uniform('C0', 0.1, 1)
         },
        
         {'penalty': 'l2',
          'solver': 'lbfgs',
          'C': hp.uniform('C1', 0.1, 1)
         }]
)

In [12]:
def hyperopt_lr(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    
    params = {'solver': params['solver'], 
              'penalty': params['penalty'], 
              'C': params['C']
             }
    # используем эту комбинацию для построения модели
    model = linear_model.LogisticRegression(
        **params, 
        random_state=random_state, 
        max_iter=50)
    
    # обучаем модель
    model.fit(X, y)
   
    # применим cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [ ]:
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_lr, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )

In [96]:
print("Наилучшие значения гиперпараметров {}".format(best))

Наилучшие значения гиперпараметров {'C0': 0.8317887360487557, 'params': 0, 'penalty': 0}


In [ ]:
# Обучаем модель с наилучшими значениями гиперпараметров
best_params = {
    'solver': 'saga',
    'penalty': 'l1',
    'C': 0.8317
}
# Создаем объект класса логистическая регрессия
hyperopt_log_reg = linear_model.LogisticRegression(
    **best_params,
    max_iter = 50)

#Обучаем модель
hyperopt_log_reg.fit(X_train, y_train)

y_test_pred = hyperopt_log_reg.predict(X_test)
y_train_pred = hyperopt_log_reg.predict(X_train)


In [7]:
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print('f1_score на тренировочном наборе : {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))

f1_score на тестовом наборе: 0.79
f1_score на тренировочном наборе : 0.85


3.4 Логистическая регрессия. OPTUNA

In [8]:
import optuna

In [20]:
def optuna_lr(trial):
  # задаем пространства поиска гиперпараметров
  penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
  solver = trial.suggest_categorical('solver', ['saga', 'lbfgs'])
  if solver == 'lbfgs':
    penalty = 'l2'
  C = trial.suggest_uniform('C', 0.1, 1)

  # создаем модель
  optuna_model = linear_model.LogisticRegression(
    penalty=penalty,
    solver=solver,
    C=C,
    random_state=random_state,
    max_iter = 50)
  
  # обучаем модель
  optuna_model.fit(X_train, y_train)
  # применим cross validation
  score = cross_val_score(optuna_model, X_train, y_train, cv=5, scoring='f1', n_jobs=-1).mean()
  
  return score

In [21]:
# cоздаем объект исследования
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_lr, n_trials=50)

[I 2022-06-23 20:35:30,218] A new study created in memory with name: LogisticRegression
/home/director/anaconda3/envs/DS/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/director/anaconda3/envs/DS/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/director/anaconda3/envs/DS/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/director/anaconda3/envs/DS/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/director/anaconda3/envs/DS/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter 

In [22]:
# Создаем объект класса логистическая регрессия
optuna_log_reg = linear_model.LogisticRegression(
    **study.best_params,
    random_state=random_state,
    max_iter = 50)

#Обучаем модель
optuna_log_reg.fit(X_train, y_train)

y_test_pred = optuna_log_reg.predict(X_test)
y_train_pred = optuna_log_reg.predict(X_train)

/home/director/anaconda3/envs/DS/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [23]:
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print('f1_score на тренировочном наборе : {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))

f1_score на тестовом наборе: 0.79
f1_score на тренировочном наборе : 0.82


##### 4. Оптимизация гиперпараметров. Случайный лес

In [24]:
from sklearn.model_selection import RandomizedSearchCV

4.1 Случайный лес. GridSearchCV

In [29]:
param_grid_rf = {
    'n_estimators': [100, 200, 400],
    'criterion': ['gini', 'entropy'],
    'max_depth': [5, 10, 25],
    'min_samples_leaf': [5, 10]
}

grid_search_rf = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=random_state),
    param_grid=param_grid_rf, 
    scoring = 'f1',
    cv=5, 
    n_jobs = -1
)

grid_search_rf.fit(X_train, y_train)
y_train_pred = grid_search_rf.predict(X_train)
y_test_pred = grid_search_rf.predict(X_test)

In [32]:
print("Наилучшие значения гиперпараметров: {}".format(grid_search_rf.best_params_))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print('f1 на кросс-валидации: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))

Наилучшие значения гиперпараметров: {'criterion': 'entropy', 'max_depth': 25, 'min_samples_leaf': 5, 'n_estimators': 200}
f1_score на тестовом наборе: 0.83
f1 на кросс-валидации: 0.95


4.2 Случайный лес. RandomizedSearchCV

In [34]:
random_search_rf = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=random_state), 
    param_distributions=param_grid_rf, 
    cv=5,
    scoring='f1',
    n_jobs = -1
)  
 
random_search_rf.fit(X_train, y_train)
y_train_pred = random_search_rf.predict(X_train)
y_test_pred = random_search_rf.predict(X_test)

In [35]:
print("Наилучшие значения гиперпараметров: {}".format(random_search_rf.best_params_))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print('f1 на кросс-валидации: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))

Наилучшие значения гиперпараметров: {'n_estimators': 100, 'min_samples_leaf': 5, 'max_depth': 25, 'criterion': 'entropy'}
f1_score на тестовом наборе: 0.82
f1 на кросс-валидации: 0.95


4.3 Случайный лес. HYPEROPT

In [36]:
space_rf = {
  'n_estimators': hp.quniform('n_estimators', 100, 400, 1),
  'criterion': hp.choice('criterion', ['gini', 'entropy']),
  'max_depth' : hp.quniform('max_depth', 1, 25, 1),
  'min_samples_leaf': hp.quniform('min_samples_leaf', 5, 10, 1)
}

In [37]:
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    
    params = {
        'n_estimators': int(params['n_estimators']),
        'criterion': params['criterion'],
        'max_depth': int(params['max_depth']), 
        'min_samples_leaf': int(params['min_samples_leaf'])
    }
    
    # модель
    model = ensemble.RandomForestClassifier(
        **params, 
        random_state=random_state)
    
    # обучаем модель
    model.fit(X, y)
   
    # применим cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [39]:
trials_rf = Trials() # используется для логирования результатов

best_rf = fmin(hyperopt_rf, # наша функция 
          space=space_rf, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials_rf, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )

100%|██████████| 20/20 [04:24<00:00, 13.24s/trial, best loss: -0.8023598964806229]


In [40]:
print("Наилучшие значения гиперпараметров {}".format(best_rf))

Наилучшие значения гиперпараметров {'criterion': 1, 'max_depth': 12.0, 'min_samples_leaf': 6.0, 'n_estimators': 326.0}


In [52]:
best_params_rf = {
    'n_estimators': int(best_rf['n_estimators']),
    'criterion': 'entropy',
    'max_depth': int(best_rf['max_depth']),
    'min_samples_leaf': int(best_rf['min_samples_leaf'])
}

# Модель с лучшими параметрами
hyperopt_rf = ensemble.RandomForestClassifier(
    **best_params_rf, 
    random_state=random_state)

hyperopt_rf.fit(X_train, y_train)
y_train_pred = hyperopt_rf.predict(X_train)
y_test_pred = hyperopt_rf.predict(X_test)

In [53]:
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print('f1_score на тренировочном наборе : {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))

f1_score на тестовом наборе: 0.81
f1_score на тренировочном наборе : 0.92


4.4 Случайный лес. OPTUNA

In [54]:
def optuna_rf(trial):
  # пространство гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 400, 1)
  criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
  max_depth = trial.suggest_int('max_depth', 1, 25, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  # создаем модель
  optuna_model = ensemble.RandomForestClassifier(
        n_estimators=n_estimators,
        criterion=criterion,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf, 
        random_state=random_state,
        n_jobs=-1
    )
  
  # обучаем модель
  optuna_model.fit(X_train, y_train)
  # применим cross validation
  score = cross_val_score(optuna_model, X_train, y_train, cv=5, scoring='f1', n_jobs=-1).mean()
  
  return score

In [57]:
# cоздаем объект исследования
study_rf = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_rf.optimize(optuna_rf, n_trials=20)

[I 2022-06-23 21:50:33,627] A new study created in memory with name: RandomForestClassifier
[I 2022-06-23 21:50:48,605] Trial 0 finished with value: 0.7969066468244452 and parameters: {'n_estimators': 224, 'criterion': 'entropy', 'max_depth': 23, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.7969066468244452.
[I 2022-06-23 21:51:04,829] Trial 1 finished with value: 0.7924141148507012 and parameters: {'n_estimators': 335, 'criterion': 'gini', 'max_depth': 25, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.7969066468244452.
[I 2022-06-23 21:51:09,127] Trial 2 finished with value: 0.7771263883662256 and parameters: {'n_estimators': 100, 'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.7969066468244452.
[I 2022-06-23 21:51:16,286] Trial 3 finished with value: 0.8003191549619123 and parameters: {'n_estimators': 127, 'criterion': 'gini', 'max_depth': 16, 'min_samples_leaf': 7}. Best is trial 3 with value: 0.8003191549619123.
[I 2022-0

In [58]:
print("Наилучшие значения гиперпараметров {}".format(study_rf.best_params))

Наилучшие значения гиперпараметров {'n_estimators': 295, 'criterion': 'entropy', 'max_depth': 19, 'min_samples_leaf': 2}


In [59]:
# Создаем объект класса логистическая регрессия
optuna_rf = ensemble.RandomForestClassifier(
    **study_rf.best_params, 
    random_state=random_state
)
    
#Обучаем модель
optuna_rf.fit(X_train, y_train)

y_test_pred = optuna_rf.predict(X_test)
y_train_pred = optuna_rf.predict(X_train)

In [60]:
print('f1 на кросс-валидации:', study_rf.best_value)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print('f1_score на тренировочном наборе : {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))

f1 на кросс-валидации: {} 0.8159601371465575
f1_score на тестовом наборе: 0.82
f1_score на тренировочном наборе : 0.99
